In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import tensorflow as tf
import numpy as np
import os, time
import traceback
from datetime import datetime
from tqdm import tqdm
from infolog import log
from utils import ValueWindow

from tensorflow.errors import OutOfRangeError
from feeder_wav import Feeder
from resnet import ResNet
from vox12_hparams import train_feeder_hparams, dev_feeder_hparams, resnet_hparams, train_hparams


number gpu: 1



In [3]:
def create_model(fbanks, labels):
    with tf.variable_scope('resnet', reuse=tf.AUTO_REUSE):
        train_resnet = ResNet(resnet_hparams, fbanks, labels, 'train')
        train_resnet.build_graph()
    with tf.variable_scope('resnet', reuse=tf.AUTO_REUSE):
        eval_resnet = ResNet(resnet_hparams, fbanks, labels, 'eval')
        eval_resnet.build_graph()

    return train_resnet, eval_resnet

# pretrain model path
model_dir = 'vox12_resnet34/'

graph = tf.Graph()
with graph.as_default():
    fbanks = tf.placeholder(dtype=tf.float32, shape=(None, None, 80), name='fbanks')
    labels = tf.placeholder(dtype=tf.int32, shape=(None), name='labels')
    _, resnet = create_model(fbanks, labels)
    saver = tf.train.Saver()
    
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(graph=graph, config=config)

checkpoint_state = tf.train.get_checkpoint_state(model_dir)
checkpoint_path = checkpoint_state.model_checkpoint_path

saver.restore(sess, checkpoint_path)




Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.







INFO:tensorflow:Restoring parameters from vox12_resnet34/gvector.ckpt-300000


## Obtain speaker embedding for dataset VCTK

In [16]:
pairs = []
save_dir = '../datasets/vctk/synthesizer/embeds/'
mel_dir = '../datasets/vctk/synthesizer/mels/'
os.makedirs(save_dir, exist_ok=True)
with open('../datasets/vctk/synthesizer/all.txt') as f:
    for line in f.readlines():
        items = line.strip().split('|')
        pairs.append((mel_dir + items[1], save_dir + items[2]))

In [19]:
for mel_file, embed_file in tqdm(pairs):
    mels = np.load(mel_file)
    embedding = sess.run(resnet.gv, feed_dict={fbanks: np.expand_dims(mels, axis=0)})
    embedding = embedding.squeeze()
    np.save(embed_file, embedding)

100%|██████████| 30617/30617 [10:05<00:00, 50.52it/s]
